In [ ]:
#OLD - NOT IN USE


import pandas as pd
import numpy as np

# total daily deliveries by zone
daily_volumes = {
    "Centrum": 200,
    "Blixembosch": 120,
    "Meerhoven": 80
}

# fraction that should be delivered within same zone vs. cross‐hub.
# For simplicity, assume:
#   - 70% of each zone's deliveries originate and end in the same zone.
#   - 20% originate in that zone but go to a different zone.
#   - 10% originate from an external warehouse (we'll assign them as origin="External", dest=that zone).
within_pct = 0.70
cross_pct = 0.20
external_pct = 0.10

# size classdistribution
classA_pct = 0.80  # 80% can be delivered by robot
classB_pct = 0.20  # 20% require bike

# time windows (in minutes from midnight)
#   morning_peak: 08:00–11:00 = minutes [480, 660)
#   midday:      11:00–15:00 = minutes [660, 900)
#   afternoon:   15:00–19:00 = minutes [900, 1140)
time_windows = [
    (480, 660, 0.50),  # 50% of daily volume
    (660, 900, 0.30),  # 30%
    (900, 1140, 0.20)  # 20%
]


In [2]:
def sample_request_times(num_parcels, windows):
    """
    Distributes num_parcels across the defined windows according to their proportions.
    Returns a numpy array of request times (in minutes).
    """
    times = []
    for start_min, end_min, frac in windows:
        count = int(round(num_parcels * frac))
        # Uniformly sample within [start_min, end_min)
        sampled = np.random.randint(start_min, end_min, size=count)
        times.extend(sampled.tolist())
    # If rounding leads to fewer or more than num_parcels, adjust:
    if len(times) < num_parcels:
        # Add random times in the full day until counts match
        extra = num_parcels - len(times)
        times.extend(
            np.random.randint(windows[0][0], windows[-1][1], size=extra).tolist()
        )
    elif len(times) > num_parcels:
        # Truncate to the first num_parcels
        times = times[:num_parcels]
    # Shuffle times so they are not grouped by window
    np.random.shuffle(times)
    return np.array(times)


In [ ]:
rows = []
parcel_id = 0

for zone, total in daily_volumes.items():
    #determine counts by origin‐destination category
    same_zone_count   = int(total * within_pct)
    cross_zone_count  = int(total * cross_pct)
    external_zone_count = total - same_zone_count - cross_zone_count
    
    # origin=zone, dest=zone, size classes, request times
    times_same = sample_request_times(same_zone_count, time_windows)
    for t in times_same:
        parcel_id += 1
        size = np.random.choice(["A", "B"], p=[classA_pct, classB_pct])
        rows.append({
            "parcel_id": parcel_id,
            "origin": zone,
            "destination": zone,
            "size_class": size,
            "request_time": t
        })
    
    # we will randomly assign each of these cross‐zone parcels to one of the other two zones as destination.
    dest_choices = [z for z in daily_volumes.keys() if z != zone]
    times_cross = sample_request_times(cross_zone_count, time_windows)
    for t in times_cross:
        parcel_id += 1
        size = np.random.choice(["A", "B"], p=[classA_pct, classB_pct])
        dest = np.random.choice(dest_choices)
        rows.append({
            "parcel_id": parcel_id,
            "origin": zone,
            "destination": dest,
            "size_class": size,
            "request_time": t
        })
    
   
    # origin="External", dest=zone. ( we assume these all appear at the zone's hub at request_time.)
    times_ext = sample_request_times(external_zone_count, time_windows)
    for t in times_ext:
        parcel_id += 1
        size = np.random.choice(["A", "B"], p=[classA_pct, classB_pct])
        rows.append({
            "parcel_id": parcel_id,
            "origin": "External",
            "destination": zone,
            "size_class": size,
            "request_time": t
        })


demand_df = pd.DataFrame(rows)
demand_df = demand_df.sample(frac=1, random_state=42).reset_index(drop=True)

#csv
demand_df.to_csv("data/raw/demand_synthetic.csv", index=False)
demand_df.head(10)


,parcel_id,origin,destination,size_class,request_time
0,210,Blixembosch,Blixembosch,A,609
1,281,Blixembosch,Blixembosch,A,553
2,34,Centrum,Centrum,A,954
3,211,Blixembosch,Blixembosch,A,928
4,94,Centrum,Centrum,A,482
5,85,Centrum,Centrum,A,528
6,330,Meerhoven,Meerhoven,A,485
7,95,Centrum,Centrum,A,498
8,267,Blixembosch,Blixembosch,A,515
9,127,Centrum,Centrum,A,1058
